# Molecular dynamics simulations with the Lennard-Jones potential

## Setup

The first thing you will need to do is install the [ASE package](https://wiki.fysik.dtu.dk/ase/) (for Atomic Simulation Environment).  First, make sure you are in the correct Python virtual environment.  If you have been installing packages with `pip`, you can just run:

    pip install ase

and it should install it into the correct environment.  Make sure it works by running the cell below:

In [1]:
import ase

If you get errors at this stage, please ask for help.

**Note for Anaconda/conda users**: The ASE package is not available through Anaconda's default distribution channels.  It is, however, available from [conda-forge](https://conda-forge.org/packages/).  You can use the following installation command (again, make sure you are in the correct environment):

    conda install -c conda-forge --strict-channel-priority ase

and try the `import` command above.

### Other imports

In [2]:
import numpy as np
from matplotlib import pyplot as plt

### Viewing atomic structures

It is important that we visualize atomic structures and their evolution in time, so that we can _see_ the (qualitative) behaviour of a system.  ASE provides a few ways to do this.  The most full-featured and user-friendly one is the [ASE GUI](https://wiki.fysik.dtu.dk/ase/ase/gui/gui.html), accessible from the command line with:

    ase gui <structure_file.xyz>

where `<structure_file.xyz>` is a file containing the structures you want to view.

You can also view structures directly in the notebook environment.  This can be very useful for quick checking of structures without interrupting your workflow.  To do this, you first have to import ASE's `view()` function:

In [12]:
from ase.visualize import view

Now, if you just call `view(structure)` on an atomic structure, you will see a visualization directly in the notebook.  Note that this visualization is still a bit basic; you can get more features by installing the `nglview` plugin:

    pip install nglview
or

    conda install -c conda-forge --strict-channel-priority nglview

You can now call the viewer function like: `view(structure, viewer='ngl')` and you will get a more interactive structure viewer.  Try it out below!

## Initial configuration

Let's start with an _initial configuration_ of atoms.  I've prepared a starting structure (a crystal of 32 Ar atoms in the FCC close-packed configuration) using the code below, but you can just load the structure from the file included with this tutorial.

In [6]:
from ase.build import bulk, make_supercell
starting_unit = bulk('Ar', 'fcc', a=5.0, cubic=True)
starting_config = make_supercell(starting_unit, 2*np.eye(3, dtype=int))

In [14]:
import ase.io

In [15]:
ase.io.write('starting_config.xyz', starting_config)

----

In [16]:
starting_config = ase.io.read('starting_config.xyz')

In [17]:
# Note: This may not work if you haven't gotten nglview installed or configured correctly
view(starting_config, viewer='ngl')

## Defining the potential

Now, we will define the _potential energy surface_ that determines how the atoms interact.  In some ways, this is the most important ingredient in an atomistic simulation!

As discussed, we will use the Lennard-Jones potential here, defined by the equation:
$$
u(r) = 4\varepsilon \left(\left(\frac{r_0}{r}\right)^{12} - \left(\frac{r_0}{r}^{6}\right)\right)
$$
applied between all pairs of atoms within some _maximum distance_ of each other.  This maximum distance is another important parameter of the potential, and it is typically called the "cutoff".

To calculate this potential for any atomic configuration, we will use ASE's built-in potential _calculator_:

In [18]:
from ase.calculators.lj import LennardJones

In [19]:
?LennardJones

Init signature: LennardJones(**kwargs)
Docstring:     
Lennard Jones potential calculator

see https://en.wikipedia.org/wiki/Lennard-Jones_potential

The fundamental definition of this potential is a pairwise energy:

``u_ij = 4 epsilon ( sigma^12/r_ij^12 - sigma^6/r_ij^6 )``

For convenience, we'll use d_ij to refer to "distance vector" and
``r_ij`` to refer to "scalar distance". So, with position vectors `r_i`:

``r_ij = | r_j - r_i | = | d_ij |``

Therefore:

``d r_ij / d d_ij = + d_ij / r_ij``
``d r_ij / d d_i  = - d_ij / r_ij``

The derivative of u_ij is:

::

    d u_ij / d r_ij
    = (-24 epsilon / r_ij) ( 2 sigma^12/r_ij^12 - sigma^6/r_ij^6 )

We can define a "pairwise force"

``f_ij = d u_ij / d d_ij = d u_ij / d r_ij * d_ij / r_ij``

The terms in front of d_ij are combined into a "general derivative".

``du_ij = (d u_ij / d d_ij) / r_ij``

We do this for convenience: `du_ij` is purely scalar The pairwise force is:

``f_ij = du_ij * d_ij``

The total force on an atom is:

``f_

Take a moment to check out the documentation.  This is also available online in the [ASE documentation page](https://wiki.fysik.dtu.dk/ase/ase/calculators/others.html#lennard-jones).

We will need to initialize it with a few parameters -- besides the cutoff radius, we need values for $\varepsilon$ and $r_0$.  Let's use the values proposed in this publication: [J. A. White, _J. Chem. Phys._ **111**, 9352-9355 (1999)]().

The proposed values are: $r_0 = 3.345\,\text{Å}$ and $\varepsilon = 125.7 k_B$, where $k_B$ is Boltzmann's constant.

But hang on -- we need to make sure to convert these to the units that ASE uses!  Luckily, ASE already uses Ångström for length units.  For the energy units, we just use the `ase.units` module to find the appropriate conversion:

In [20]:
ase.units.kB

8.617330337217213e-05

So, putting it together, we initialize the potential as follows:

In [21]:
lj_argon_calc = LennardJones(sigma=3.345, epsilon=125.7*ase.units.kB, rc=10.0, smooth=True)

(I've somewhat arbitrarily chosen a cutoff radius of 10 Å here because that's the length of our cell.  You can play around with different cutoff values later, although be warned that the effect on the results can be quite subtle.)

## Doing something with the potential

Now that we've defined the potential, let's do something useful with it!  The simplest thing would be an energy minimization, where we try to find the structure that minimizes the energy of this potential energy surface.  Note that this structure will generally depend on the potential and its parameters!

In [22]:
starting_config.set_calculator(lj_argon_calc)

/tmp/ipykernel_780710/392841954.py:1: FutureWarning: Please use atoms.calc = calc
  starting_config.set_calculator(lj_argon_calc)


In [25]:
from ase.optimize import BFGS

In [28]:
from ase.filters import ExpCellFilter

In [33]:
# Slightly annoying extra step we need in order to optimize the cell (volume) in addition to the positions
atoms_cell = ExpCellFilter(starting_config, hydrostatic_strain=True)
opt = BFGS(atoms_cell)
opt.run(fmax=0.005)

      Step     Time          Energy          fmax
BFGS:    0 15:51:53       -2.778298        0.000596


/tmp/ipykernel_780710/4288921084.py:2: DeprecationWarning: Use FrechetCellFilter for better convergence w.r.t. cell variables.
  atoms_cell = ExpCellFilter(starting_config, hydrostatic_strain=True)


np.True_

In [31]:
atoms_cell

In [32]:
starting_config

Atoms(symbols='Ar32', pbc=True, cell=[[10.347299727835516, 7.186239713009969e-21, 9.447929792997826e-21], [-8.985169089170857e-18, 10.347299727835516, 2.9693821976474967e-21], [1.820059191444276e-19, -4.154090480839192e-20, 10.347299727835516]], calculator=LennardJones(...))

### Question

What is the _cell parameter_ of the resulting, optimized structure?  How does it compare to the experimental value (for solid argon)?

## Run dynamics

Okay, now we get to the fun part!  Let's run some actual simulations and see the atoms move!

In [35]:
import ase.md

First, we need an "integrator" that actually solves the equations of motion.  Velocity Verlet is a good choice:

In [36]:
vv = ase.md.VelocityVerlet(atoms=starting_config, timestep=1.0, trajectory='trial_traj.xyz')

And now we just run it for 50 timesteps and see what happens!

In [38]:
vv.run(50)

True

Finally, try loading the output in the `ase gui` and see if we got anything...

----
SPOILER ALERT



...nothing happened.  That's because we haven't initialized the _velocities_ on the atoms, so they're not moving!  Let's set them up with a nice initial, "thermal" velocity.

In [39]:
from ase.md import velocitydistribution

In [40]:
velocitydistribution.MaxwellBoltzmannDistribution(starting_config, temperature_K=20)

In [41]:
vv.run(50)

True